In [1]:
import requests
import pandas as pd
import numpy as np
import time as time

import seaborn as sns
import matplotlib.pyplot as plt

In [2]:

#allow all columns to be seen. See show all columns
pd.set_option('display.max_columns', None)

# Source:https://stackoverflow.com/questions/47022070/display-all-dataframe-columns-in-a-jupyter-python-notebook/47022213 

#all text to be seen
pd.set_option('display.max_colwidth', None)

#https://stackoverflow.com/questions/25351968/how-to-display-full-non-truncated-dataframe-information-in-html-when-convertin 

# Read in the Twitter Data

In [3]:
tweets = pd.read_csv('./tweets_data/tweets_predictions.csv')
tweets.head(2)

,Index,User,Text,Date,Favorites,Retweets,Mentions,HashTags,streets,highways,exits,markers,road_closed,predictions
0,591,iembot_lch,"At 4:45 PM CDT, Burns Point [St. Mary Co, LA] PUBLIC reports STORM SURGE. STORM SURGE BEGINNING TO COVER PORTIONS OF HIGHWAY 317. PHOTOS VIA SOCIAL MEDIA. https://mesonet.agron.iastate.edu/lsr/#LCH/202008262145/202008262145",2020-08-27 21:47:48+00:00,0,0,NaN,NaN,[],['highway 317'],[],[],0.0,0.0
1,646,TotalTrafficBTR,Downed tree cleared in #WestBatonRouge on I 10 WB after LA-415/Lobdell/Exit 151 and before LA-77/Rosedale/Grosse Tete/Exit 139 #traffic http://bit.ly/12oyfoo,2020-08-27 11:50:43+00:00,0,0,NaN,#WestBatonRouge #traffic,[],[],"['exit 151', 'exit 139']",[],NaN,0.0


In [4]:
tweets.shape

(1037, 14)

# Prep the Data

## Convert the data into a `query_string` column

In [104]:
#Take streets, highways, exits, and markers and put them in one column
tweets['query'] = tweets['streets'] + tweets['highways'] + tweets['exits'] + tweets['markers']

In [105]:
#remove brackets
tweets['query'] = [val.replace('[]','') for val in tweets['query'].values]
tweets['query'] = [val.replace('][',',') for val in tweets['query'].values]
tweets['query'] = [val.replace(']','') for val in tweets['query'].values]
tweets['query'] = [val.replace('[','') for val in tweets['query'].values]

In [106]:
query = []

#url encode the column
for val in tweets['query']:
    val = val.replace(',','%2C+')
    val = val.replace(' ','+')
    val = val.replace("'",'')
    val = val.replace("",'')
    s = ''.join(val)
    query.append(s)

In [107]:
#create the query_string column
tweets['query_string'] = query
tweets.head(2)

,Index,User,Text,Date,Favorites,Retweets,Mentions,HashTags,streets,highways,exits,markers,road_closed,predictions,query,query_string
0,591,iembot_lch,"At 4:45 PM CDT, Burns Point [St. Mary Co, LA] PUBLIC reports STORM SURGE. STORM SURGE BEGINNING TO COVER PORTIONS OF HIGHWAY 317. PHOTOS VIA SOCIAL MEDIA. https://mesonet.agron.iastate.edu/lsr/#LCH/202008262145/202008262145",2020-08-27 21:47:48+00:00,0,0,NaN,NaN,[],['highway 317'],[],[],0.0,0.0,'highway 317',highway+317
1,646,TotalTrafficBTR,Downed tree cleared in #WestBatonRouge on I 10 WB after LA-415/Lobdell/Exit 151 and before LA-77/Rosedale/Grosse Tete/Exit 139 #traffic http://bit.ly/12oyfoo,2020-08-27 11:50:43+00:00,0,0,NaN,#WestBatonRouge #traffic,[],[],"['exit 151', 'exit 139']",[],NaN,0.0,"'exit 151', 'exit 139'",exit+151%2C++exit+139


# GET Lat/Lng from HERE API

`queryScore` – a value from `0` to `1` representing the percentage of the input that matched the returned address. It is equal to `1` if all input tokens were recognized and matched. If the input query contains any additional information – name of the person, phone number or a hint to ring twice – the `queryScore` will be low, but the result may be correct. But low queryScore may indicate that the result does not match some relevant part of the address in the input query.

`fieldScore` – values from `0` to `1` indicating how good the result field matches to the corresponding part of the query. If all the fields that are important for addressing in the country are matched with high quality, one can assume, that queryScore is low because of the additional non-address information. And the result is still confident.

Resource: https://developer.here.com/documentation/geocoding-search-api/dev_guide/topics-api/code-geocode-hybrid.html

In [5]:
api_key = '<add api key here>' #commented out for privacy reasons

In [7]:
# get the lat and long for all values

#there must be a 'query_string' column in the data frame
def get_lat_lng(df,state):
    lat = []
    lng = []
    query_score = []
    field_score = []

    for row in df['query_string']:
        res=requests.get('https://geocode.search.hereapi.com/v1/geocode?q=' + row + '&qq=state='+ state + '&apiKey=' + api_key) #create the api get request
        time.sleep(2) #add a sleep to not hit servers too hard
        json = res.json() #extract json

        #if no value is returned, just put 0.0
        try:
            #append to lat
            lat.append(json['items'][0]['position']['lat'])

            #append to long
            lng.append(json['items'][0]['position']['lng'])

            #append to query_score
            query_score.append(json['items'][0]['scoring']['queryScore'])

            #append to field_score
            field_score.append(json['items'][0]['scoring']['fieldScore'])

        except IndexError:
            #append to lat
            lat.append(0.0)

            #append to long
            lng.append(0.0)

            #append to query_score
            query_score.append(0.0)

            #append to field_score
            field_score.append(0.0)
        
        except KeyError:
            #append to lat
            lat.append(0.0)

            #append to long
            lng.append(0.0)

            #append to query_score
            query_score.append(0.0)

            #append to field_score
            field_score.append(0.0)
    
    #Add columns to the dataframe
    df['lat'] = lat
    df['lng'] = lng
    df['query_score'] = query_score
    df['field_score'] = field_score
    
    return df


# GET lat/lng from Twitter Data

In [110]:
time0 = time.time() #start timer

tweets_with_geo = get_lat_lng(tweets,'Louisana') #run function

print(f'Time to run: {(time.time() - time0) / 60} mins')

Time to run: 43.55741340319316 mins


In [112]:
tweets.to_csv('./GeoCoding Data/tweets_with_geo.csv',index=False)

# LADOT FEED

In [6]:
ladot_feed = pd.read_csv('./LADOTD Data/road_closures.csv')
ladot_feed.head(3)

,date,report
0,9/11/2020,"I-10 Eastbound and Westbound from MM 103 (I-49) to MM 109 (LA 328 – Breaux Bridge), Lafayette/St. Martin Parish – Intermittent Short Term Lane Closure"
1,9/11/2020,"I-10 WB from MM 103 to MM 104, St. Martin Parish, Lane Closure H.003003"
2,9/11/2020,I-10- Partial Road Closure - Project #: H.011024 - St. Tammany Parish (Oak Harbor Blvd.)


## Create the query string

In [16]:
ladot_feed['query_string'] = [val.replace(' ','+') for val in ladot_feed['report']] #here api will process out the extra words
ladot_feed.head(3) #check work

,date,closed,report,query_string
0,9/11/2020,NaN,"I-10 Eastbound and Westbound from MM 103 (I-49) to MM 109 (LA 328 – Breaux Bridge), Lafayette/St. Martin Parish – Intermittent Short Term Lane Closure","I-10+Eastbound+and+Westbound+from+MM+103+(I-49)+to+MM+109+(LA+328+–+Breaux+Bridge),+Lafayette/St.+Martin+Parish+–+Intermittent+Short+Term+Lane+Closure"
1,9/11/2020,NaN,"I-10 WB from MM 103 to MM 104, St. Martin Parish, Lane Closure H.003003","I-10+WB+from+MM+103+to+MM+104,+St.+Martin+Parish,+Lane+Closure+H.003003"
2,9/11/2020,NaN,I-10- Partial Road Closure - Project #: H.011024 - St. Tammany Parish (Oak Harbor Blvd.),I-10-+Partial+Road+Closure+-+Project+#:+H.011024+-+St.+Tammany+Parish+(Oak+Harbor+Blvd.)


## Run the function to get lat/lng

In [17]:
time0 = time.time()

ladot_feed_with_geo =  get_lat_lng(ladot_feed,'Louisiana')

print(f'Time to run: {(time.time() - time0) / 60} mins')

Time to run: 30.976319766044618 mins


In [20]:
ladot_feed_with_geo.to_csv('./GeoCoding Data/ladot_feed_with_geo.csv',index=False)